# 1. Data cleaning

## 1.1 Read CSV into DataFrame

### 1.1.1 Instagram

In [22]:
import datetime
import re
import pandas as pd

def combine_dataframe(platform, reference_list):
    frames = []
    if platform == 'instagram':
        for insta_handle in reference_list:
            filename = f'{platform}/{insta_handle[1:]}_comments.csv'
            df = pd.read_csv(filename).drop('Unnamed: 0', axis=1)
            df.likes = df.likes.apply(lambda x: x if type(x) == float else x[0] if x[0].isdigit() else 0)
            df['celeb'] = reference_list[insta_handle]
            frames.append(df)        
        df = pd.concat(frames)
        df['platform'] = platform
        df = df[['username', 'datetime', 'comment', 'likes', 'celeb', 'platform']]
    
    elif platform == 'twitter':
        for name in reference_list:
            df = pd.read_csv(f'{platform}/{name}').drop('Unnamed: 0', axis=1)
            df = pd.DataFrame({'username': df.username, 'datetime': df.date, 'comment': df.text, 'likes': df.retweet_count, 'celeb': df.name_of_celeb, 'platform': platform})
            frames.append(df)
        df = pd.concat(frames)
    
    elif platform == 'youtube':
        for name in reference_list:
            try:  
                df = pd.read_excel(f'{platform}/{name}.xlsx', sheet_name=[1,2,3])
            except:
                continue
            df = pd.concat(df.values())
            df = pd.DataFrame({'username': df.Name, 'datetime': df.Time, 'comment': df.Comment, 'likes': df.Likes, 'celeb': name, 'platform': platform})
            frames.append(df)
        df = pd.concat(frames)
    
    elif platform == 'reddit':
        for name in reference_list:
            if name == 'Janelle Monáe':
                name = 'Janelle Monae'
            df = pd.read_csv(f'{platform}/{platform}_{name}.csv')
            df = pd.DataFrame({'username': df.SubmissionID, 'datetime': df.Timestamp, 'comment': df.Comment, 'likes': 0, 'celeb': name, 'platform': platform})
            frames.append(df)
        df = pd.concat(frames)
    
    elif platform == 'facebook':
        df = pd.read_csv(f'{platform}/FB_Celebs.csv')
        df = pd.DataFrame({'username': df.object_id, 'datetime': df.created_time, 'comment': df.message, 'likes': df.like_count, 'celeb': df.celeb, 'platform': platform})

    return df


def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

def remove_symb(text):
    cleaned_text = re.sub(r'[,./?!]', ' ', text)
    return cleaned_text

def remove_url(stringliteral):
    return re.sub(r'http\S+', '', stringliteral)

def change_like(text):
    if type(text) == int or type(text) == float:
        return int(text)
    if text.lower() == 'reply':
        return 0
    if 'likes' in text:
        return int(text.replace('likes', '').replace(',', ''))
    return 1

def datetime_converter(text):
    if len(text) > 10:
        text = text[:10]
    try:
        return datetime.datetime.strptime(text, "%Y-%m-%d")
    except:
        return datetime.datetime.strptime(text, "%d/%m/%Y")

In [4]:
all_insta_handle_list = list(pd.read_excel(f'List_of_Celebrities.xlsx').iloc[:50]['Instagram Username'])
all_celeb = list(map(str.strip, pd.read_excel(f'List_of_Celebrities.xlsx').iloc[:50]['Name']))
insta_dict = {all_insta_handle_list[i]:all_celeb[i] for i in range(50)}

In [5]:
df_insta = combine_dataframe('instagram', insta_dict)

In [6]:
df_insta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 652534 entries, 0 to 29246
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   username  652520 non-null  object
 1   datetime  652534 non-null  object
 2   comment   652531 non-null  object
 3   likes     652525 non-null  object
 4   celeb     652534 non-null  object
 5   platform  652534 non-null  object
dtypes: object(6)
memory usage: 34.8+ MB


### 1.1.2 Facebook

In [7]:
df_facebook = combine_dataframe('facebook', all_celeb)

C:\Users\for_i\AppData\Local\Temp\ipykernel_23532\2904187273.py:46: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{platform}/FB_Celebs.csv')


In [8]:
df_facebook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36390 entries, 0 to 36389
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   username  36390 non-null  object 
 1   datetime  34823 non-null  object 
 2   comment   31302 non-null  object 
 3   likes     34094 non-null  float64
 4   celeb     36341 non-null  object 
 5   platform  36390 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.7+ MB


### 1.1.3 Twitter

In [9]:
csv_filename_list = ['2020_01_15.csv', '2020_02_07.csv', '2020_03_30.csv', '2020_04_23.csv', '2020_05_05.csv', '2020_06_28.csv', '2020_07_02.csv', '2020_08_22.csv', '2020_09_09.csv', '2020_10_12.csv', '2020_12_17.csv']
df_twitter = combine_dataframe('twitter', csv_filename_list)

In [10]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122985 entries, 0 to 10808
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   username  122985 non-null  object
 1   datetime  122985 non-null  object
 2   comment   122985 non-null  object
 3   likes     122985 non-null  int64 
 4   celeb     122985 non-null  object
 5   platform  122985 non-null  object
dtypes: int64(1), object(5)
memory usage: 6.6+ MB


### 1.1.4 Youtube

In [11]:
df_youtube = combine_dataframe('youtube', all_celeb)

In [12]:
df_youtube.info()

<class 'pandas.core.frame.DataFrame'>
Index: 702725 entries, 0 to 43
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   username  510342 non-null  object
 1   datetime  514481 non-null  object
 2   comment   514477 non-null  object
 3   likes     514481 non-null  object
 4   celeb     702725 non-null  object
 5   platform  702725 non-null  object
dtypes: object(6)
memory usage: 37.5+ MB


### 1.1.5 Reddit

In [14]:
df_reddit = combine_dataframe('reddit', all_celeb)

In [15]:
df_reddit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 315419 entries, 0 to 13032
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   username  315419 non-null  object
 1   datetime  315419 non-null  object
 2   comment   315415 non-null  object
 3   likes     315419 non-null  int64 
 4   celeb     315419 non-null  object
 5   platform  315419 non-null  object
dtypes: int64(1), object(5)
memory usage: 16.8+ MB


## 1.3 Combine all 5 dataframe into 1

In [16]:
df = pd.concat([df_insta, df_facebook, df_twitter, df_youtube, df_reddit]).reset_index()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830053 entries, 0 to 1830052
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   index     int64 
 1   username  object
 2   datetime  object
 3   comment   object
 4   likes     object
 5   celeb     object
 6   platform  object
dtypes: int64(1), object(6)
memory usage: 97.7+ MB


## 1.4 Clean Null Values

In [18]:
df.isnull().sum()

index            0
username    192397
datetime    189811
comment     193343
likes       190549
celeb           49
platform         0
dtype: int64

In [19]:
df.dropna(subset=['username', 'comment', 'celeb'], inplace=True)
df.fillna(value={"likes": 0}, inplace=True)


In [20]:
df.isnull().sum()

index       0
username    0
datetime    0
comment     0
likes       0
celeb       0
platform    0
dtype: int64

## 1.5 Clean Comments

In [23]:
df.comment = df.comment.apply(lambda x: '' if type(x) != str else x)
df.comment = df.comment.apply(striphtml)
df.comment = df.comment.apply(remove_url)
df['comment'] = df['comment'].apply(remove_symb)
df.likes = df.likes.apply(change_like)
df = df[df.comment != '']

## 1.6 Clean datetime

In [24]:
df.datetime = df.datetime.apply(datetime_converter)

In [25]:
df.head()

,index,username,datetime,comment,likes,celeb,platform
0,0,pierrearden_team_gelord,2020-09-15,Hey Ben great feed we'd like to organise a c...,1,Ben Platt,instagram
1,1,ophelielaplante,2020-09-19,@_shxniex_,1,Ben Platt,instagram
2,2,dr_wsutton,2020-09-27,So fucking sexy @bensplatt,0,Ben Platt,instagram
3,3,pwpmccormack,2020-10-26,Cute,0,Ben Platt,instagram
4,4,bby.joker_,2020-10-29,@lordemusic 🥀✨ @ellamai @palomamami @sza,0,Ben Platt,instagram


In [27]:
new_columns = ["TOXICITY", "SEVERE_TOXICITY", "SEXUALLY_EXPLICIT", "THREAT", "INSULT", "IDENTITY_ATTACK"]
for column in new_columns:
    df[column] = 0

df.drop('index', axis=1)

,username,datetime,comment,likes,celeb,platform,TOXICITY,SEVERE_TOXICITY,SEXUALLY_EXPLICIT,THREAT,INSULT,IDENTITY_ATTACK
0,pierrearden_team_gelord,2020-09-15,Hey Ben great feed we'd like to organise a c...,1,Ben Platt,instagram,0,0,0,0,0,0
1,ophelielaplante,2020-09-19,@_shxniex_,1,Ben Platt,instagram,0,0,0,0,0,0
2,dr_wsutton,2020-09-27,So fucking sexy @bensplatt,0,Ben Platt,instagram,0,0,0,0,0,0
3,pwpmccormack,2020-10-26,Cute,0,Ben Platt,instagram,0,0,0,0,0,0
4,bby.joker_,2020-10-29,@lordemusic 🥀✨ @ellamai @palomamami @sza,0,Ben Platt,instagram,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1830048,kn22lk,2020-12-30,Yeah there’s no way the public unemployment fi...,0,Joe Biden,reddit,0,0,0,0,0,0
1830049,kn22lk,2020-12-30,Even if dems win both races in Georgia the rep...,0,Joe Biden,reddit,0,0,0,0,0,0
1830050,kn22lk,2020-12-30,Trump's fault is the shitload of people that d...,0,Joe Biden,reddit,0,0,0,0,0,0
1830051,kn22lk,2020-12-30,A Snafu,0,Joe Biden,reddit,0,0,0,0,0,0


In [28]:
df.to_csv('comments_not_label.csv', index=False, quoting=2)